In [5]:
import pandas as pd
import numpy as np

In [74]:
#Check preprocessing

preprocessed_available_pokemons = catalog.load("preprocessed_available_pokemons")
preprocessed_battle_results = catalog.load("preprocessed_battle_results")
all_pokemons = catalog.load("all_pokemons")
battle_results = catalog.load("battle_results")

# master = create_master_table(preprocessed_shuttles, preprocessed_companies, reviews)
# master.head()

2020-05-30 12:34:57,307 - kedro.io.data_catalog - INFO - Loading data from `preprocessed_available_pokemons` (CSVDataSet)...
2020-05-30 12:34:57,374 - kedro.io.data_catalog - INFO - Loading data from `preprocessed_battle_results` (CSVDataSet)...
2020-05-30 12:35:02,188 - kedro.io.data_catalog - INFO - Loading data from `all_pokemons` (CSVDataSet)...
2020-05-30 12:35:02,206 - kedro.io.data_catalog - INFO - Loading data from `battle_results` (CSVDataSet)...


ValueError: Cannot index with multidimensional key

In [82]:
#Check for NaN values
print(battle_results.isnull().values.any())
print(battle_results.isnull().sum().sum())

print(all_pokemons.isnull().values.any())
print(all_pokemons.isnull().sum().sum())

False
0
True
86


In [3]:
preprocessed_battle_results.head()
preprocessed_available_pokemons.head()
# all_pokemons.head()

,Name_1,Level_1,Price_1,HP_1,Attack_1,Defense_1,Sp_Atk_1,Sp_Def_1,Speed_1,Legendary_1,SelectedPokemonID
0,Abra,4,346,32,22,17,115,61,99,0,1
1,Abra,26,614,80,38,26,197,103,170,0,2
2,Abra,33,775,102,48,33,247,131,214,0,3
3,Abra,45,937,129,56,39,298,158,257,0,4
4,Abra,46,963,133,58,40,306,162,264,0,5


In [6]:
pd.set_option("display.max_rows", 151, "display.max_columns", None)
print(all_pokemons)
pd.set_option("display.max_rows", 10, "display.max_columns", None)

      ID        Name    Type_1    Type_2
0      1   Bulbasaur     Grass    Poison
1      2     Ivysaur     Grass    Poison
2      3    Venusaur     Grass    Poison
3      4  Charmander      Fire       NaN
4      5  Charmeleon      Fire       NaN
5      6   Charizard      Fire    Flying
6      7    Squirtle     Water       NaN
7      8   Wartortle     Water       NaN
8      9   Blastoise     Water       NaN
9     10    Caterpie       Bug       NaN
10    11     Metapod       Bug       NaN
11    12  Butterfree       Bug    Flying
12    13      Weedle       Bug    Poison
13    14      Kakuna       Bug    Poison
14    15    Beedrill       Bug    Poison
15    16      Pidgey    Normal    Flying
16    17   Pidgeotto    Normal    Flying
17    18     Pidgeot    Normal    Flying
18    19     Rattata    Normal       NaN
19    20    Raticate    Normal       NaN
20    21     Spearow    Normal    Flying
21    22      Fearow    Normal    Flying
22    23       Ekans    Poison       NaN
23    24       A

In [343]:
# print(preprocessed_available_pokemons)

In [19]:
# Test master table for integrating type1,type2 in battle_results

def create_battle_results_with_types(
    all_pokemons: pd.DataFrame, battle_results: pd.DataFrame) -> pd.DataFrame:
    """Integrates Type_1 and Type_2 from 'all_pokemons' into battle_results.
       Gets projected onto Type_1_1 and Type_2_1 for Pokemon1 and Type_1_2 and Type_2_2 for Pokemon2.
       'NaN' is replaced with '0'
       Also integrates IDs for Pokemons => ID_1, ID_2
       P1, P2 used for 'Pokemon 1' and 'Pokemon 2'

        Args:
            all_pokemons: Preprocessed data for All Pokemons.
            battle_results: Preprocessed data for Battle Results.
        Returns:
            battle_results_with_types

    """
    #Avoid Nidorana a second time (IDs 29 and 32) in all_pokemons
    all_pokemons = all_pokemons[all_pokemons['ID'] != 32] 
    
    ## Pokemon1
    # Merge battle_results with all_pokemons based on the Name of P1
    battle_results_with_types_P1 = battle_results.merge(all_pokemons, left_on="Name_1", right_on="Name")
    # Rename Type_1 and Type_2 to Type_1_1 and Type_2_1
    battle_results_with_types_P1.rename(columns={'Type_1': 'Type_1_1', 'Type_2': 'Type_2_1', 'ID':'ID_1'}, inplace=True)
    
    #Drop 'Name' as it is already saved in 'Name_1'
    battle_results_with_types_P1.drop(["Name"], axis=1, inplace=True)
    
    
    ## Adding Pokemon2
    # Merge battle_results_with_types_P1 with all_pokemons based on the Name of P2
    battle_results_with_types = battle_results_with_types_P1.merge(all_pokemons, left_on="Name_2", right_on="Name")
    
    # Rename Type_1 and Type_2 to Type_1_1 and Type_2_1
    battle_results_with_types.rename(columns={'Type_1': 'Type_1_2', 'Type_2': 'Type_2_2', 'ID':'ID_2'}, inplace=True)
    
    #Drop 'Name' as it is already saved in 'Name_2'
    battle_results_with_types.drop(["Name"], axis=1, inplace=True)
    
    battle_results_with_types = battle_results_with_types.fillna('')
    return battle_results_with_types

In [20]:
battle_results_with_types = create_battle_results_with_types(all_pokemons, preprocessed_battle_results)

In [21]:
print(len(preprocessed_battle_results))
print(len(battle_results_with_types))
print(len(battle_results_with_types)-len(preprocessed_battle_results))
print(len(preprocessed_battle_results['Name_1'].unique().tolist()))

# Here I found out that number of ID != number of names in all_pokemons because of Nidorana appearing twice (genderwise)
print(len(all_pokemons['ID'].unique().tolist()))
print(len(all_pokemons['Name'].unique().tolist()))

2719163
2719163
0
144
151
150


In [22]:
#preprocessed_available_pokemons[preprocessed_available_pokemons['Name_1']=='Nidorana']

In [24]:
pd.set_option("display.max_rows", 20, "display.min_rows", 10, "display.max_columns", None)
# battle_results_with_types['Name_1'].unique().tolist()

battle_results_with_types.loc[battle_results_with_types['Name_1'] == 'Nidorana']

,Name_1,Level_1,Price_1,HP_1,Attack_1,Defense_1,Sp_Atk_1,Sp_Def_1,Speed_1,Legendary_1,Name_2,Level_2,Price_2,HP_2,Attack_2,Defense_2,Sp_Atk_2,Sp_Def_2,Speed_2,Legendary_2,WeatherAndTime,BattleResult,ID_1,Type_1_1,Type_2_1,ID_2,Type_1_2,Type_2_2
12278,Nidorana,28,517,137,78,90,70,70,72,0,Pinsir,40,1401,228,346,270,146,187,224,0,Unknown,-213,29,Poison,0,127,Bug,0
12279,Nidorana,32,632,122,129,88,88,88,117,0,Pinsir,72,2874,481,707,554,295,380,457,0,Night,-481,29,Poison,0,127,Bug,0
12280,Nidorana,81,2272,600,350,386,311,311,314,0,Pinsir,89,4913,821,1208,947,504,652,781,0,Windy,-653,29,Poison,0,127,Bug,0
12281,Nidorana,7,320,67,54,59,46,46,48,0,Pinsir,6,561,73,140,112,62,78,96,0,Night,-45,29,Poison,0,127,Bug,0
12282,Nidorana,33,614,162,94,107,83,83,85,0,Pinsir,61,2204,368,543,424,227,292,350,0,Rain,-368,29,Poison,0,127,Bug,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2712813,Nidorana,27,523,101,107,73,73,73,96,0,Metapod,65,945,324,75,220,103,103,120,0,Unknown,-286,29,Poison,0,11,Bug,0
2712814,Nidorana,68,1625,430,250,277,222,222,224,0,Metapod,12,280,86,25,68,32,32,37,0,Windy,430,29,Poison,0,11,Bug,0
2712815,Nidorana,80,2151,424,442,295,295,295,400,0,Metapod,93,1806,623,145,417,196,196,229,0,Rain,204,29,Poison,0,11,Bug,0
2712816,Nidorana,43,813,213,125,140,111,111,113,0,Metapod,84,1531,529,123,353,166,166,194,0,Windy,-454,29,Poison,0,11,Bug,0


In [25]:
len(battle_results.loc[battle_results['Name_1'] == 'Nidorana'])

36910

In [26]:
battle_results_with_types[['Name_1','ID_1','Type_1_1', 'Type_2_1','Name_2','ID_2','Type_1_2', 'Type_2_2']].head()

,Name_1,ID_1,Type_1_1,Type_2_1,Name_2,ID_2,Type_1_2,Type_2_2
0,Metapod,11,Bug,0,Pinsir,127,Bug,0
1,Metapod,11,Bug,0,Pinsir,127,Bug,0
2,Metapod,11,Bug,0,Pinsir,127,Bug,0
3,Metapod,11,Bug,0,Pinsir,127,Bug,0
4,Metapod,11,Bug,0,Pinsir,127,Bug,0


In [27]:
# Test master table for feature hashing type1,type2 in battle_results

from sklearn.feature_extraction import FeatureHasher

def create_battle_results_with_hashed_types(
    battle_results: pd.DataFrame) -> pd.DataFrame:
    """Feature hashes 'Type_1' and 'Type_2' and 'WeatherAndTime' from 'battle_results_with_types'.
       Example for labeling: Type_1_1 => Type_1_1_a, Type_1_1_b, Type_1_1_c, Type_1_1_d, Type_1_1_e
       number of features: 5 (labeled with a,b,c,d,e)
       For transparency, original Types/WeatherandTime are not dropped in DataFrame

        Args:
            battle_results: Preprocessed data for Battle Results with types
        Returns:
            battle_results_with_hashed_types_weather
    """
    
    ##Feature Hasher for Type
    h_type = FeatureHasher(n_features=5, input_type='string')
    
    #Transform Types of Pokemon1: Type1_1, Type2_1 and of Pokemon2: Type1_2, Type2_2
    dtype1_1 = h_type.fit_transform(battle_results["Type_1_1"])
    dtype2_1 = h_type.fit_transform(battle_results["Type_2_1"])
    dtype1_2 = h_type.fit_transform(battle_results["Type_1_2"])
    dtype2_2 = h_type.fit_transform(battle_results["Type_2_2"])
    
    #Transform into Pandas DataFrame and rename labels
    dtype1_1 = pd.DataFrame(data=dtype1_1.toarray())
    dtype1_1.rename(columns={0: 'Type_1_1_a', 1: 'Type_1_1_b', 2: 'Type_1_1_c', 3: 'Type_1_1_d', 4: 'Type_1_1_e'}, inplace=True)
    dtype2_1 = pd.DataFrame(data=dtype2_1.toarray())
    dtype2_1.rename(columns={0: 'Type_2_1_a', 1: 'Type_2_1_b', 2: 'Type_2_1_c', 3: 'Type_2_1_d', 4: 'Type_2_1_e'}, inplace=True)
    dtype1_2 = pd.DataFrame(data=dtype1_2.toarray())
    dtype1_2.rename(columns={0: 'Type_1_2_a', 1: 'Type_1_2_b', 2: 'Type_1_2_c', 3: 'Type_1_2_d', 4: 'Type_1_2_e'}, inplace=True)
    dtype2_2 = pd.DataFrame(data=dtype2_2.toarray())
    dtype2_2.rename(columns={0: 'Type_2_2_a', 1: 'Type_2_2_b', 2: 'Type_2_2_c', 3: 'Type_2_2_d', 4: 'Type_2_2_e'}, inplace=True)

    
    ##Feature Hasher for WeatherandTime
    h_wt = FeatureHasher(n_features=5, input_type='string')
    dwt = h_wt.fit_transform(battle_results["WeatherAndTime"])
    dwt = pd.DataFrame(data=dwt.toarray())
    dwt.rename(columns={0: 'WeatherAndTime_a', 1: 'WeatherAndTime_b', 2: 'WeatherAndTime_c', 3: 'WeatherAndTime_d', 4: 'WeatherAndTime_e'}, inplace=True)
    
    
    #(No) Drop of Type_1_1, ... and WeatherandTime column.
    #battle_results = battle_results.drop(columns = ["WeatherAndTime", "Type_1_1", "Type_2_1", "Type_1_2", "Type_2_2"])
    
    #Concatenate battle_results with hashed features
    battle_results_hashed = pd.concat([battle_results, dtype1_1, dtype1_2, dtype2_1, dtype2_2, dwt], axis=1)
    
    return battle_results_hashed

/Users/lucazurmuehle/anaconda3/envs/pokehack/lib/python3.6/site-packages/_pytest/mark/structures.py:443: DeprecationWarning: The usage of `cmp` is deprecated and will be removed on or after 2021-06-01.  Please use `eq` and `order` instead.
  @attr.s(cmp=False, hash=False)


In [28]:
battle_results_hashed = create_battle_results_with_hashed_types(battle_results_with_types)

In [32]:
battle_results_hashed[['Name_1','Type_1_2','Type_1_1_a', 'Type_1_1_b', 'Type_1_1_c', 'Type_1_1_d', 'Type_1_1_e']]

,Name_1,Type_1_2,Type_1_1_a,Type_1_1_b,Type_1_1_c,Type_1_1_d,Type_1_1_e
0,Metapod,Bug,0.0,0.0,-1.0,0.0,0.0
1,Metapod,Bug,0.0,0.0,-1.0,0.0,0.0
2,Metapod,Bug,0.0,0.0,-1.0,0.0,0.0
3,Metapod,Bug,0.0,0.0,-1.0,0.0,0.0
4,Metapod,Bug,0.0,0.0,-1.0,0.0,0.0
...,...,...,...,...,...,...,...
2719158,Slowpoke,Bug,2.0,0.0,0.0,0.0,-1.0
2719159,Slowpoke,Bug,2.0,0.0,0.0,0.0,-1.0
2719160,Slowpoke,Bug,2.0,0.0,0.0,0.0,-1.0
2719161,Slowpoke,Bug,2.0,0.0,0.0,0.0,-1.0


In [33]:
battle_results_hashed.keys()

Index(['Name_1', 'Level_1', 'Price_1', 'HP_1', 'Attack_1', 'Defense_1',
       'Sp_Atk_1', 'Sp_Def_1', 'Speed_1', 'Legendary_1', 'Name_2', 'Level_2',
       'Price_2', 'HP_2', 'Attack_2', 'Defense_2', 'Sp_Atk_2', 'Sp_Def_2',
       'Speed_2', 'Legendary_2', 'WeatherAndTime', 'BattleResult', 'ID_1',
       'Type_1_1', 'Type_2_1', 'ID_2', 'Type_1_2', 'Type_2_2', 'Type_1_1_a',
       'Type_1_1_b', 'Type_1_1_c', 'Type_1_1_d', 'Type_1_1_e', 'Type_1_2_a',
       'Type_1_2_b', 'Type_1_2_c', 'Type_1_2_d', 'Type_1_2_e', 'Type_2_1_a',
       'Type_2_1_b', 'Type_2_1_c', 'Type_2_1_d', 'Type_2_1_e', 'Type_2_2_a',
       'Type_2_2_b', 'Type_2_2_c', 'Type_2_2_d', 'Type_2_2_e',
       'WeatherAndTime_a', 'WeatherAndTime_b', 'WeatherAndTime_c',
       'WeatherAndTime_d', 'WeatherAndTime_e'],
      dtype='object')

In [34]:
# Create master table with Attack/Defense ratios


def create_battle_results_AD(
    battle_results: pd.DataFrame) -> pd.DataFrame:
    """Creates additional features 'Attack1/Defense2', 'Attack2/Defense1'. Same for Special Attack/Special Defense

        Args:
            battle_results: Preprocessed data for Battle Results
        Returns:
            battle_results: Battle Results with A/D
    """
    
    battle_results['A1/D2'] = battle_results['Attack_1']/battle_results['Defense_2']
    battle_results['A2/D1'] = battle_results['Attack_2']/battle_results['Defense_1']
    battle_results['Sp_A1/Sp_D2'] = battle_results['Sp_Atk_1']/battle_results['Sp_Def_2']
    battle_results['Sp_A2/Sp_D1'] = battle_results['Sp_Atk_2']/battle_results['Sp_Def_1']

    return battle_results

In [35]:
battle_results_AD = create_battle_results_AD(battle_results_hashed)

2020-05-30 12:08:16,836 - numexpr.utils - INFO - NumExpr defaulting to 4 threads.


In [37]:
battle_results_AD.head()

,Name_1,Level_1,Price_1,HP_1,Attack_1,Defense_1,Sp_Atk_1,Sp_Def_1,Speed_1,Legendary_1,Name_2,Level_2,Price_2,HP_2,Attack_2,Defense_2,Sp_Atk_2,Sp_Def_2,Speed_2,Legendary_2,WeatherAndTime,BattleResult,ID_1,Type_1_1,Type_2_1,ID_2,Type_1_2,Type_2_2,Type_1_1_a,Type_1_1_b,Type_1_1_c,Type_1_1_d,Type_1_1_e,Type_1_2_a,Type_1_2_b,Type_1_2_c,Type_1_2_d,Type_1_2_e,Type_2_1_a,Type_2_1_b,Type_2_1_c,Type_2_1_d,Type_2_1_e,Type_2_2_a,Type_2_2_b,Type_2_2_c,Type_2_2_d,Type_2_2_e,WeatherAndTime_a,WeatherAndTime_b,WeatherAndTime_c,WeatherAndTime_d,WeatherAndTime_e,A1/D2,A2/D1,Sp_A1/Sp_D2,Sp_A2/Sp_D1
0,Metapod,30,441,150,36,103,47,47,58,0,Pinsir,47,1589,262,393,306,164,211,253,0,Windy,-262,11,Bug,0,127,Bug,0,0.0,0.0,-1.0,0.0,0.0,0.0,0.0,-1.0,0.0,0.0,0.0,0.0,0.0,0.0,-1.0,0.0,0.0,0.0,0.0,-1.0,0.0,-1.0,0.0,0.0,0.0,0.117647,3.815534,0.222749,3.489362
1,Metapod,15,297,92,26,73,33,33,40,0,Pinsir,89,4913,821,1208,947,504,652,781,0,Windy,-821,11,Bug,0,127,Bug,0,0.0,0.0,-1.0,0.0,0.0,0.0,0.0,-1.0,0.0,0.0,0.0,0.0,0.0,0.0,-1.0,0.0,0.0,0.0,0.0,-1.0,0.0,-1.0,0.0,0.0,0.0,0.027455,16.547945,0.050613,15.272727
2,Metapod,97,2007,693,161,463,218,218,254,0,Pinsir,67,2477,414,610,477,254,328,394,0,Night,-132,11,Bug,0,127,Bug,0,0.0,0.0,-1.0,0.0,0.0,0.0,0.0,-1.0,0.0,0.0,0.0,0.0,0.0,0.0,-1.0,0.0,0.0,0.0,0.0,-1.0,-1.0,-1.0,-2.0,0.0,-1.0,0.337526,1.317495,0.664634,1.165138
3,Metapod,62,882,303,70,205,96,96,112,0,Pinsir,37,1288,206,319,249,135,172,207,0,Sunshine,-110,11,Bug,0,127,Bug,0,0.0,0.0,-1.0,0.0,0.0,0.0,0.0,-1.0,0.0,0.0,0.0,0.0,0.0,0.0,-1.0,0.0,0.0,0.0,0.0,-1.0,-1.0,-1.0,1.0,-1.0,-2.0,0.281124,1.556098,0.558140,1.406250
4,Metapod,85,1548,535,124,357,168,168,196,0,Pinsir,84,4322,722,1063,833,444,573,687,0,Unknown,-664,11,Bug,0,127,Bug,0,0.0,0.0,-1.0,0.0,0.0,0.0,0.0,-1.0,0.0,0.0,0.0,0.0,0.0,0.0,-1.0,0.0,0.0,0.0,0.0,-1.0,-1.0,-2.0,0.0,1.0,-3.0,0.148860,2.977591,0.293194,2.642857


In [46]:
battle_results_AD.loc[battle_results_AD['Type_2_1'] ==  '0']

,Name_1,Level_1,Price_1,HP_1,Attack_1,Defense_1,Sp_Atk_1,Sp_Def_1,Speed_1,Legendary_1,Name_2,Level_2,Price_2,HP_2,Attack_2,Defense_2,Sp_Atk_2,Sp_Def_2,Speed_2,Legendary_2,WeatherAndTime,BattleResult,ID_1,Type_1_1,Type_2_1,ID_2,Type_1_2,Type_2_2,Type_1_1_a,Type_1_1_b,Type_1_1_c,Type_1_1_d,Type_1_1_e,Type_1_2_a,Type_1_2_b,Type_1_2_c,Type_1_2_d,Type_1_2_e,Type_2_1_a,Type_2_1_b,Type_2_1_c,Type_2_1_d,Type_2_1_e,Type_2_2_a,Type_2_2_b,Type_2_2_c,Type_2_2_d,Type_2_2_e,WeatherAndTime_a,WeatherAndTime_b,WeatherAndTime_c,WeatherAndTime_d,WeatherAndTime_e,A1/D2,A2/D1,Sp_A1/Sp_D2,Sp_A2/Sp_D1
0,Metapod,30,441,150,36,103,47,47,58,0,Pinsir,47,1589,262,393,306,164,211,253,0,Windy,-262,11,Bug,0,127,Bug,0,0.0,0.0,-1.0,0.0,0.0,0.0,0.0,-1.0,0.0,0.0,0.0,0.0,0.0,0.0,-1.0,0.0,0.0,0.0,0.0,-1.0,0.0,-1.0,0.0,0.0,0.0,0.117647,3.815534,0.222749,3.489362
1,Metapod,15,297,92,26,73,33,33,40,0,Pinsir,89,4913,821,1208,947,504,652,781,0,Windy,-821,11,Bug,0,127,Bug,0,0.0,0.0,-1.0,0.0,0.0,0.0,0.0,-1.0,0.0,0.0,0.0,0.0,0.0,0.0,-1.0,0.0,0.0,0.0,0.0,-1.0,0.0,-1.0,0.0,0.0,0.0,0.027455,16.547945,0.050613,15.272727
2,Metapod,97,2007,693,161,463,218,218,254,0,Pinsir,67,2477,414,610,477,254,328,394,0,Night,-132,11,Bug,0,127,Bug,0,0.0,0.0,-1.0,0.0,0.0,0.0,0.0,-1.0,0.0,0.0,0.0,0.0,0.0,0.0,-1.0,0.0,0.0,0.0,0.0,-1.0,-1.0,-1.0,-2.0,0.0,-1.0,0.337526,1.317495,0.664634,1.165138
3,Metapod,62,882,303,70,205,96,96,112,0,Pinsir,37,1288,206,319,249,135,172,207,0,Sunshine,-110,11,Bug,0,127,Bug,0,0.0,0.0,-1.0,0.0,0.0,0.0,0.0,-1.0,0.0,0.0,0.0,0.0,0.0,0.0,-1.0,0.0,0.0,0.0,0.0,-1.0,-1.0,-1.0,1.0,-1.0,-2.0,0.281124,1.556098,0.558140,1.406250
4,Metapod,85,1548,535,124,357,168,168,196,0,Pinsir,84,4322,722,1063,833,444,573,687,0,Unknown,-664,11,Bug,0,127,Bug,0,0.0,0.0,-1.0,0.0,0.0,0.0,0.0,-1.0,0.0,0.0,0.0,0.0,0.0,0.0,-1.0,0.0,0.0,0.0,0.0,-1.0,-1.0,-2.0,0.0,1.0,-3.0,0.148860,2.977591,0.293194,2.642857
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2717876,Squirtle,1,314,44,48,65,50,64,43,0,Metapod,52,721,248,57,168,78,78,92,0,Unknown,-248,7,Water,0,11,Bug,0,2.0,0.0,0.0,0.0,-1.0,0.0,0.0,-1.0,0.0,0.0,0.0,0.0,0.0,0.0,-1.0,0.0,0.0,0.0,0.0,-1.0,-1.0,-2.0,0.0,1.0,-3.0,0.285714,0.876923,0.641026,1.218750
2717877,Squirtle,21,558,94,80,114,83,113,74,0,Metapod,32,456,155,37,106,49,49,60,0,Night,-8,7,Water,0,11,Bug,0,2.0,0.0,0.0,0.0,-1.0,0.0,0.0,-1.0,0.0,0.0,0.0,0.0,0.0,0.0,-1.0,0.0,0.0,0.0,0.0,-1.0,-1.0,-1.0,-2.0,0.0,-1.0,0.754717,0.324561,1.693878,0.433628
2717878,Squirtle,22,587,99,84,120,87,119,78,0,Metapod,37,499,172,40,116,53,53,65,0,Unknown,-25,7,Water,0,11,Bug,0,2.0,0.0,0.0,0.0,-1.0,0.0,0.0,-1.0,0.0,0.0,0.0,0.0,0.0,0.0,-1.0,0.0,0.0,0.0,0.0,-1.0,-1.0,-2.0,0.0,1.0,-3.0,0.724138,0.333333,1.641509,0.445378
2717879,Squirtle,11,417,67,61,86,63,85,55,0,Metapod,23,365,121,31,86,39,39,49,0,Unknown,-43,7,Water,0,11,Bug,0,2.0,0.0,0.0,0.0,-1.0,0.0,0.0,-1.0,0.0,0.0,0.0,0.0,0.0,0.0,-1.0,0.0,0.0,0.0,0.0,-1.0,-1.0,-2.0,0.0,1.0,-3.0,0.709302,0.360465,1.615385,0.458824


In [48]:
weakness_pokemons = catalog.load("weakness_pokemons")

2020-05-30 12:16:44,040 - kedro.io.data_catalog - INFO - Loading data from `weakness_pokemons` (CSVDataSet)...


In [50]:
# print(weakness_pokemons)
# weakness_pokemons.loc[weakness_pokemons['Types'] == 'Water']['Fire']
# battle_results_AD.keys()
weakness_pokemons

,Types,Normal,Fire,Water,Electric,Grass,Ice,Fighting,Poison,Ground,Flying,Psychic,Bug,Rock,Ghost,Dragon,Dark,Steel,Fairy
0,Normal,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,0.6,0.6,1.0,1.0,0.6,1.0
1,Fire,1.0,0.6,0.6,1.0,50.0,6.0,1.0,1.0,1.0,1.0,1.0,6.0,0.6,1.0,0.6,1.0,6.0,1.0
2,Water,1.0,20.0,0.6,1.0,0.6,1.0,1.0,1.0,6.0,1.0,1.0,1.0,6.0,1.0,0.6,1.0,1.0,1.0
3,Electric,1.0,1.0,6.0,0.6,0.6,1.0,1.0,1.0,0.6,6.0,1.0,1.0,1.0,1.0,0.6,1.0,1.0,1.0
4,Grass,1.0,0.6,6.0,1.0,0.6,1.0,1.0,0.6,6.0,0.6,1.0,0.6,6.0,1.0,0.6,1.0,0.6,1.0
5,Ice,1.0,0.6,0.6,1.0,6.0,0.6,1.0,1.0,6.0,6.0,1.0,1.0,1.0,1.0,6.0,1.0,0.6,1.0
6,Fighting,6.0,1.0,1.0,1.0,1.0,6.0,1.0,0.6,1.0,0.6,0.6,0.6,6.0,0.6,1.0,6.0,6.0,0.6
7,Poison,1.0,1.0,1.0,1.0,6.0,1.0,1.0,0.6,0.6,1.0,1.0,1.0,0.6,0.6,1.0,1.0,0.6,6.0
8,Ground,1.0,6.0,1.0,6.0,0.6,1.0,1.0,6.0,1.0,0.6,1.0,0.6,6.0,1.0,1.0,1.0,6.0,1.0
9,Flying,1.0,1.0,1.0,0.6,6.0,1.0,6.0,1.0,1.0,1.0,1.0,6.0,0.6,1.0,1.0,1.0,0.6,1.0


In [51]:
# merge = pd.merge(battle_results_AD,weakness_pokemons,left_on=['Type_1_1','Type_1_2'],right_on=['Types', weakness_pokemons.columns])

In [52]:
# merge.head()

In [69]:
# pd.set_option("display.max_rows", 10, "display.max_columns", None)

#battle_results_weakness = battle_results_AD.loc[weakness_pokemons['Types'] == battle_results_AD['Type_1_1']].apply()#[battle_results_AD['Type_1_1']]
#m = weakness_pokemons['Types'] == battle_results_AD['Type_1_1']


#weakness_pokemons.loc[weakness_pokemons['Types'] == 'Water']['Fire']


#types = weakness_pokemons['Types']

#for i in range(len(types)):
#weakness_pokemons.loc[weakn
                      
#np.where(weakness_pokemons['Types'] == battle_results_AD['Type_1_1']) #[battle_results_AD['Type_1_2']] for i

#print(next(battle_results_AD['Type_1_1'].iterrows())[1])


# print(types[0])
#(weakness_pokemons['Types'] == 'Water')
# battle_results_AD['Type_1_1'][1]

#weakness_pokemons.where(m)
# battle_results_AD['EFF1_2'] = battle_results_AD['Type_1_1'].map(weakness_pokemons)
# battle_results_AD['Type_1_1']
# weakness_pokemons
                      
weakness_long = weakness_pokemons.melt(id_vars="Types")
weakness_long.columns = ["Type_P1", "Type_P2", "value"]
weakness_long

#Eff_11
battle_results_with_weakness = battle_results_AD.merge(weakness_long, left_on=["Type_1_1","Type_1_2"], right_on=["Type_P1","Type_P2"])
battle_results_with_weakness.rename(columns={'value': 'value_1_1ab'}, inplace=True)
battle_results_with_weakness.drop(['Type_P1', 'Type_P2'], axis=1, inplace=True)

battle_results_with_weakness = battle_results_with_weakness.merge(weakness_long, left_on=["Type_1_2","Type_1_1"], right_on=["Type_P1","Type_P2"])
battle_results_with_weakness.rename(columns={'value': 'value_1_1ba'}, inplace=True)
battle_results_with_weakness.drop(['Type_P1', 'Type_P2'], axis=1, inplace=True)

# battle_results_with_weakness = battle_results_with_weakness.fillna(1)

battle_results_with_weakness['Eff_11'] = battle_results_with_weakness['value_1_1ab']/battle_results_with_weakness['value_1_1ba']




In [88]:
# battle_results_with_weakness.loc[battle_results_with_weakness['Eff_11'].isnull()

print(battle_results_with_weakness['Type_2_2'].isnull().values.any())
print(battle_results_with_weakness['Type_2_2'].isnull().sum())

print(battle_results_with_weakness.isnull().sum().sum())

False
0
0


In [111]:
#Eff_12
battle_results_with_weakness_2 = battle_results_with_weakness.merge(weakness_long, left_on=["Type_1_1","Type_2_2"], right_on=["Type_P1","Type_P2"], how='left')
battle_results_with_weakness_2.rename(columns={'value': 'value_1_2ab'}, inplace=True)
battle_results_with_weakness_2.drop(['Type_P1', 'Type_P2'], axis=1, inplace=True)

battle_results_with_weakness_2 = battle_results_with_weakness_2.merge(weakness_long, left_on=["Type_2_2","Type_1_1"], right_on=["Type_P1","Type_P2"], how='left')
battle_results_with_weakness_2.rename(columns={'value': 'value_2_1ba'}, inplace=True)
battle_results_with_weakness_2.drop(['Type_P1', 'Type_P2'], axis=1, inplace=True)

battle_results_with_weakness_2 = battle_results_with_weakness_2.fillna(1)

battle_results_with_weakness_2['Eff_12'] = battle_results_with_weakness_2['value_1_2ab']/battle_results_with_weakness_2['value_2_1ba']

# np.where(battle_results_with_weakness_2['value_1_2ab'] == 'NaN', 1)

In [112]:
#df['your column name'].isnull().values.any()

# battle_results_with_weakness_2.loc[battle_results_with_weakness_2['Type_2_2'].isnull()]
print(battle_results_with_weakness_2['Type_2_2'].isnull().values.any())
print(battle_results_with_weakness_2['Type_2_2'].isnull().sum())

print(battle_results_with_weakness_2['Type_1_2'].isnull().values.any())
print(battle_results_with_weakness_2['Name_1'].isnull().sum())

print(battle_results_with_weakness_2.isnull().sum().sum())

False
0
False
0
0


In [113]:
battle_results_with_weakness_2.loc[battle_results_with_weakness['Type_2_2'] == '0']

,Name_1,Level_1,Price_1,HP_1,Attack_1,Defense_1,Sp_Atk_1,Sp_Def_1,Speed_1,Legendary_1,Name_2,Level_2,Price_2,HP_2,Attack_2,Defense_2,Sp_Atk_2,Sp_Def_2,Speed_2,Legendary_2,WeatherAndTime,BattleResult,ID_1,Type_1_1,Type_2_1,ID_2,Type_1_2,Type_2_2,Type_1_1_a,Type_1_1_b,Type_1_1_c,Type_1_1_d,Type_1_1_e,Type_1_2_a,Type_1_2_b,Type_1_2_c,Type_1_2_d,Type_1_2_e,Type_2_1_a,Type_2_1_b,Type_2_1_c,Type_2_1_d,Type_2_1_e,Type_2_2_a,Type_2_2_b,Type_2_2_c,Type_2_2_d,Type_2_2_e,WeatherAndTime_a,WeatherAndTime_b,WeatherAndTime_c,WeatherAndTime_d,WeatherAndTime_e,A1/D2,A2/D1,Sp_A1/Sp_D2,Sp_A2/Sp_D1,value_1_1ab,value_1_1ba,Eff_11,value_1_2ab,value_2_1ba,Eff_12
0,Metapod,30,441,150,36,103,47,47,58,0,Pinsir,47,1589,262,393,306,164,211,253,0,Windy,-262,11,Bug,0,127,Bug,0,0.0,0.0,-1.0,0.0,0.0,0.0,0.0,-1.0,0.0,0.0,0.0,0.0,0.0,0.0,-1.0,0.0,0.0,0.0,0.0,-1.0,0.0,-1.0,0.0,0.0,0.0,0.117647,3.815534,0.222749,3.489362,1.0,1.0,1.0,1.0,1.0,1.0
1,Metapod,15,297,92,26,73,33,33,40,0,Pinsir,89,4913,821,1208,947,504,652,781,0,Windy,-821,11,Bug,0,127,Bug,0,0.0,0.0,-1.0,0.0,0.0,0.0,0.0,-1.0,0.0,0.0,0.0,0.0,0.0,0.0,-1.0,0.0,0.0,0.0,0.0,-1.0,0.0,-1.0,0.0,0.0,0.0,0.027455,16.547945,0.050613,15.272727,1.0,1.0,1.0,1.0,1.0,1.0
2,Metapod,97,2007,693,161,463,218,218,254,0,Pinsir,67,2477,414,610,477,254,328,394,0,Night,-132,11,Bug,0,127,Bug,0,0.0,0.0,-1.0,0.0,0.0,0.0,0.0,-1.0,0.0,0.0,0.0,0.0,0.0,0.0,-1.0,0.0,0.0,0.0,0.0,-1.0,-1.0,-1.0,-2.0,0.0,-1.0,0.337526,1.317495,0.664634,1.165138,1.0,1.0,1.0,1.0,1.0,1.0
3,Metapod,62,882,303,70,205,96,96,112,0,Pinsir,37,1288,206,319,249,135,172,207,0,Sunshine,-110,11,Bug,0,127,Bug,0,0.0,0.0,-1.0,0.0,0.0,0.0,0.0,-1.0,0.0,0.0,0.0,0.0,0.0,0.0,-1.0,0.0,0.0,0.0,0.0,-1.0,-1.0,-1.0,1.0,-1.0,-2.0,0.281124,1.556098,0.558140,1.406250,1.0,1.0,1.0,1.0,1.0,1.0
4,Metapod,85,1548,535,124,357,168,168,196,0,Pinsir,84,4322,722,1063,833,444,573,687,0,Unknown,-664,11,Bug,0,127,Bug,0,0.0,0.0,-1.0,0.0,0.0,0.0,0.0,-1.0,0.0,0.0,0.0,0.0,0.0,0.0,-1.0,0.0,0.0,0.0,0.0,-1.0,-1.0,-2.0,0.0,1.0,-3.0,0.148860,2.977591,0.293194,2.642857,1.0,1.0,1.0,1.0,1.0,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2685565,Slowpoke,87,2834,849,586,586,350,350,113,0,Poliwag,75,2178,312,360,287,287,287,645,0,Windy,705,79,Water,Psychic,60,Water,0,2.0,0.0,0.0,0.0,-1.0,2.0,0.0,0.0,0.0,-1.0,-1.0,-2.0,-2.0,0.0,0.0,0.0,0.0,0.0,0.0,-1.0,0.0,-1.0,0.0,0.0,0.0,2.041812,0.614334,1.219512,0.820000,0.6,0.6,1.0,1.0,1.0,1.0
2685566,Slowpoke,22,531,157,109,109,67,67,22,0,Poliwag,17,491,66,83,66,66,66,144,0,Night,121,79,Water,Psychic,60,Water,0,2.0,0.0,0.0,0.0,-1.0,2.0,0.0,0.0,0.0,-1.0,-1.0,-2.0,-2.0,0.0,0.0,0.0,0.0,0.0,0.0,-1.0,-1.0,-1.0,-2.0,0.0,-1.0,1.651515,0.761468,1.015152,0.985075,0.6,0.6,1.0,1.0,1.0,1.0
2685567,Slowpoke,90,3122,935,646,646,385,385,125,0,Poliwag,18,498,67,84,67,67,67,146,0,Sunshine,926,79,Water,Psychic,60,Water,0,2.0,0.0,0.0,0.0,-1.0,2.0,0.0,0.0,0.0,-1.0,-1.0,-2.0,-2.0,0.0,0.0,0.0,0.0,0.0,0.0,-1.0,-1.0,-1.0,1.0,-1.0,-2.0,9.641791,0.130031,5.746269,0.174026,0.6,0.6,1.0,1.0,1.0,1.0
2685568,Slowpoke,8,381,110,79,79,48,48,17,0,Poliwag,52,1282,180,212,170,170,170,380,0,Rain,-180,79,Water,Psychic,60,Water,0,2.0,0.0,0.0,0.0,-1.0,2.0,0.0,0.0,0.0,-1.0,-1.0,-2.0,-2.0,0.0,0.0,0.0,0.0,0.0,0.0,-1.0,1.0,-1.0,0.0,0.0,0.0,0.464706,2.683544,0.282353,3.541667,0.6,0.6,1.0,1.0,1.0,1.0


In [115]:
#Eff_21
battle_results_with_weakness_3 = battle_results_with_weakness_2.merge(weakness_long, left_on=["Type_2_1","Type_1_2"], right_on=["Type_P1","Type_P2"], how='left')
battle_results_with_weakness_3.rename(columns={'value': 'value_2_1ab'}, inplace=True)
battle_results_with_weakness_3.drop(['Type_P1', 'Type_P2'], axis=1, inplace=True)

battle_results_with_weakness_3 = battle_results_with_weakness_3.merge(weakness_long, left_on=["Type_1_2","Type_2_1"], right_on=["Type_P1","Type_P2"], how='left')
battle_results_with_weakness_3.rename(columns={'value': 'value_1_2ba'}, inplace=True)
battle_results_with_weakness_3.drop(['Type_P1', 'Type_P2'], axis=1, inplace=True)

battle_results_with_weakness_3 = battle_results_with_weakness_3.fillna(1)

battle_results_with_weakness_3['Eff_21'] = battle_results_with_weakness_3['value_2_1ab']/battle_results_with_weakness_3['value_1_2ba']


In [121]:
battle_results_with_weakness_3.loc[battle_results_with_weakness_2['Type_2_1'] == '0']

,Name_1,Level_1,Price_1,HP_1,Attack_1,Defense_1,Sp_Atk_1,Sp_Def_1,Speed_1,Legendary_1,Name_2,Level_2,Price_2,HP_2,Attack_2,Defense_2,Sp_Atk_2,Sp_Def_2,Speed_2,Legendary_2,WeatherAndTime,BattleResult,ID_1,Type_1_1,Type_2_1,ID_2,Type_1_2,Type_2_2,Type_1_1_a,Type_1_1_b,Type_1_1_c,Type_1_1_d,Type_1_1_e,Type_1_2_a,Type_1_2_b,Type_1_2_c,Type_1_2_d,Type_1_2_e,Type_2_1_a,Type_2_1_b,Type_2_1_c,Type_2_1_d,Type_2_1_e,Type_2_2_a,Type_2_2_b,Type_2_2_c,Type_2_2_d,Type_2_2_e,WeatherAndTime_a,WeatherAndTime_b,WeatherAndTime_c,WeatherAndTime_d,WeatherAndTime_e,A1/D2,A2/D1,Sp_A1/Sp_D2,Sp_A2/Sp_D1,value_1_1ab,value_1_1ba,Eff_11,value_1_2ab,value_2_1ba,Eff_12,value_2_1ab,value_1_2ba,Eff_21
0,Metapod,30,441,150,36,103,47,47,58,0,Pinsir,47,1589,262,393,306,164,211,253,0,Windy,-262,11,Bug,0,127,Bug,0,0.0,0.0,-1.0,0.0,0.0,0.0,0.0,-1.0,0.0,0.0,0.0,0.0,0.0,0.0,-1.0,0.0,0.0,0.0,0.0,-1.0,0.0,-1.0,0.0,0.0,0.0,0.117647,3.815534,0.222749,3.489362,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0
1,Metapod,15,297,92,26,73,33,33,40,0,Pinsir,89,4913,821,1208,947,504,652,781,0,Windy,-821,11,Bug,0,127,Bug,0,0.0,0.0,-1.0,0.0,0.0,0.0,0.0,-1.0,0.0,0.0,0.0,0.0,0.0,0.0,-1.0,0.0,0.0,0.0,0.0,-1.0,0.0,-1.0,0.0,0.0,0.0,0.027455,16.547945,0.050613,15.272727,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0
2,Metapod,97,2007,693,161,463,218,218,254,0,Pinsir,67,2477,414,610,477,254,328,394,0,Night,-132,11,Bug,0,127,Bug,0,0.0,0.0,-1.0,0.0,0.0,0.0,0.0,-1.0,0.0,0.0,0.0,0.0,0.0,0.0,-1.0,0.0,0.0,0.0,0.0,-1.0,-1.0,-1.0,-2.0,0.0,-1.0,0.337526,1.317495,0.664634,1.165138,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0
3,Metapod,62,882,303,70,205,96,96,112,0,Pinsir,37,1288,206,319,249,135,172,207,0,Sunshine,-110,11,Bug,0,127,Bug,0,0.0,0.0,-1.0,0.0,0.0,0.0,0.0,-1.0,0.0,0.0,0.0,0.0,0.0,0.0,-1.0,0.0,0.0,0.0,0.0,-1.0,-1.0,-1.0,1.0,-1.0,-2.0,0.281124,1.556098,0.558140,1.406250,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0
4,Metapod,85,1548,535,124,357,168,168,196,0,Pinsir,84,4322,722,1063,833,444,573,687,0,Unknown,-664,11,Bug,0,127,Bug,0,0.0,0.0,-1.0,0.0,0.0,0.0,0.0,-1.0,0.0,0.0,0.0,0.0,0.0,0.0,-1.0,0.0,0.0,0.0,0.0,-1.0,-1.0,-2.0,0.0,1.0,-3.0,0.148860,2.977591,0.293194,2.642857,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2717966,Squirtle,40,905,159,129,183,132,182,120,0,Slowpoke,77,2194,658,454,454,270,270,88,0,Windy,-636,7,Water,0,79,Water,Psychic,2.0,0.0,0.0,0.0,-1.0,2.0,0.0,0.0,0.0,-1.0,0.0,0.0,0.0,0.0,-1.0,-1.0,-2.0,-2.0,0.0,0.0,0.0,-1.0,0.0,0.0,0.0,0.284141,2.480874,0.488889,1.483516,0.6,0.6,1.0,1.0,1.0,1.0,1.0,1.0,1.0
2717967,Squirtle,50,1284,225,182,260,187,259,171,0,Slowpoke,36,792,237,162,162,99,99,33,0,Rain,161,7,Water,0,79,Water,Psychic,2.0,0.0,0.0,0.0,-1.0,2.0,0.0,0.0,0.0,-1.0,0.0,0.0,0.0,0.0,-1.0,-1.0,-2.0,-2.0,0.0,0.0,1.0,-1.0,0.0,0.0,0.0,1.123457,0.623077,1.888889,0.382239,0.6,0.6,1.0,1.0,1.0,1.0,1.0,1.0,1.0
2717968,Squirtle,64,1934,338,274,392,281,391,258,0,Slowpoke,58,1414,425,291,291,175,175,57,0,Rain,110,7,Water,0,79,Water,Psychic,2.0,0.0,0.0,0.0,-1.0,2.0,0.0,0.0,0.0,-1.0,0.0,0.0,0.0,0.0,-1.0,-1.0,-2.0,-2.0,0.0,0.0,1.0,-1.0,0.0,0.0,0.0,0.941581,0.742347,1.605714,0.447570,0.6,0.6,1.0,1.0,1.0,1.0,1.0,1.0,1.0
2717969,Squirtle,94,4205,738,597,849,612,846,563,0,Slowpoke,6,372,107,77,77,47,47,17,0,Sunshine,738,7,Water,0,79,Water,Psychic,2.0,0.0,0.0,0.0,-1.0,2.0,0.0,0.0,0.0,-1.0,0.0,0.0,0.0,0.0,-1.0,-1.0,-2.0,-2.0,0.0,0.0,-1.0,-1.0,1.0,-1.0,-2.0,7.753247,0.090695,13.021277,0.055556,0.6,0.6,1.0,1.0,1.0,1.0,1.0,1.0,1.0


In [119]:
# Test whether there are Null values
# print(battle_results_with_weakness_3['Type_2_1'].isnull().values.any())
# print(battle_results_with_weakness_3['Type_2_1'].isnull().sum())

# print(battle_results_with_weakness_3['Type_1_2'].isnull().values.any())
# print(battle_results_with_weakness_3['Name_1'].isnull().sum())

print(battle_results_with_weakness_3.isnull().sum().sum())

0


In [120]:
#Eff_22
battle_results_with_weakness_4 = battle_results_with_weakness_3.merge(weakness_long, left_on=["Type_2_1","Type_2_2"], right_on=["Type_P1","Type_P2"], how='left')
battle_results_with_weakness_4.rename(columns={'value': 'value_2_2ab'}, inplace=True)
battle_results_with_weakness_4.drop(['Type_P1', 'Type_P2'], axis=1, inplace=True)

battle_results_with_weakness_4 = battle_results_with_weakness_4.merge(weakness_long, left_on=["Type_2_2","Type_2_1"], right_on=["Type_P1","Type_P2"], how='left')
battle_results_with_weakness_4.rename(columns={'value': 'value_2_2ba'}, inplace=True)
battle_results_with_weakness_4.drop(['Type_P1', 'Type_P2'], axis=1, inplace=True)

battle_results_with_weakness_4 = battle_results_with_weakness_4.fillna(1)

battle_results_with_weakness_4['Eff_22'] = battle_results_with_weakness_4['value_2_2ab']/battle_results_with_weakness_4['value_2_2ba']

In [134]:
battle_results_with_weakness_4.loc[(battle_results_with_weakness_4['Type_2_1'] == 'Water') & (battle_results_with_weakness_4['Type_1_2'] == 'Fire')]

,Name_1,Level_1,Price_1,HP_1,Attack_1,Defense_1,Sp_Atk_1,Sp_Def_1,Speed_1,Legendary_1,Name_2,Level_2,Price_2,HP_2,Attack_2,Defense_2,Sp_Atk_2,Sp_Def_2,Speed_2,Legendary_2,WeatherAndTime,BattleResult,ID_1,Type_1_1,Type_2_1,ID_2,Type_1_2,Type_2_2,Type_1_1_a,Type_1_1_b,Type_1_1_c,Type_1_1_d,Type_1_1_e,Type_1_2_a,Type_1_2_b,Type_1_2_c,Type_1_2_d,Type_1_2_e,Type_2_1_a,Type_2_1_b,Type_2_1_c,Type_2_1_d,Type_2_1_e,Type_2_2_a,Type_2_2_b,Type_2_2_c,Type_2_2_d,Type_2_2_e,WeatherAndTime_a,WeatherAndTime_b,WeatherAndTime_c,WeatherAndTime_d,WeatherAndTime_e,A1/D2,A2/D1,Sp_A1/Sp_D2,Sp_A2/Sp_D1,value_1_1ab,value_1_1ba,Eff_11,value_1_2ab,value_2_1ba,Eff_12,value_2_1ab,value_1_2ba,Eff_21,value_2_2ab,value_2_2ba,Eff_22
752219,Omastar,70,2817,461,336,694,632,395,299,0,Rapidash,11,636,100,124,86,98,98,130,0,Night,461,139,Rock,Water,78,Fire,0,0.0,-1.0,-1.0,1.0,1.0,1.0,-1.0,0.0,-1.0,1.0,2.0,0.0,0.0,0.0,-1.0,0.0,0.0,0.0,0.0,-1.0,-1.0,-1.0,-2.0,0.0,-1.0,3.906977,0.178674,6.448980,0.248101,6.0,0.6,10.0,1.0,1.0,1.0,20.0,0.6,33.333333,1.0,1.0,1.0
752220,Omastar,14,681,106,82,169,154,95,75,0,Rapidash,96,4700,858,878,611,710,710,933,0,Windy,-858,139,Rock,Water,78,Fire,0,0.0,-1.0,-1.0,1.0,1.0,1.0,-1.0,0.0,-1.0,1.0,2.0,0.0,0.0,0.0,-1.0,0.0,0.0,0.0,0.0,-1.0,0.0,-1.0,0.0,0.0,0.0,0.134206,5.195266,0.216901,7.473684,6.0,0.6,10.0,1.0,1.0,1.0,20.0,0.6,33.333333,1.0,1.0,1.0
752221,Omastar,57,2154,346,259,532,484,303,230,0,Rapidash,93,4276,780,798,557,646,646,849,0,Windy,84,139,Rock,Water,78,Fire,0,0.0,-1.0,-1.0,1.0,1.0,1.0,-1.0,0.0,-1.0,1.0,2.0,0.0,0.0,0.0,-1.0,0.0,0.0,0.0,0.0,-1.0,0.0,-1.0,0.0,0.0,0.0,0.464991,1.500000,0.749226,2.132013,6.0,0.6,10.0,1.0,1.0,1.0,20.0,0.6,33.333333,1.0,1.0,1.0
752222,Omastar,66,2501,410,298,616,561,350,266,0,Rapidash,39,1188,210,225,156,180,180,237,0,Windy,410,139,Rock,Water,78,Fire,0,0.0,-1.0,-1.0,1.0,1.0,1.0,-1.0,0.0,-1.0,1.0,2.0,0.0,0.0,0.0,-1.0,0.0,0.0,0.0,0.0,-1.0,0.0,-1.0,0.0,0.0,0.0,1.910256,0.365260,3.116667,0.514286,6.0,0.6,10.0,1.0,1.0,1.0,20.0,0.6,33.333333,1.0,1.0,1.0
752223,Omastar,42,1476,236,178,364,332,209,157,0,Rapidash,57,1720,313,322,224,260,260,341,0,Rain,184,139,Rock,Water,78,Fire,0,0.0,-1.0,-1.0,1.0,1.0,1.0,-1.0,0.0,-1.0,1.0,2.0,0.0,0.0,0.0,-1.0,0.0,0.0,0.0,0.0,-1.0,1.0,-1.0,0.0,0.0,0.0,0.794643,0.884615,1.276923,1.244019,6.0,0.6,10.0,1.0,1.0,1.0,20.0,0.6,33.333333,1.0,1.0,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
764152,Omanyte,13,485,64,53,131,117,72,48,0,Charizard,68,3392,569,524,476,663,533,627,0,Windy,-569,138,Rock,Water,6,Fire,Flying,0.0,-1.0,-1.0,1.0,1.0,1.0,-1.0,0.0,-1.0,1.0,2.0,0.0,0.0,0.0,-1.0,0.0,-1.0,0.0,-1.0,0.0,0.0,-1.0,0.0,0.0,0.0,0.111345,4.000000,0.219512,9.208333,6.0,0.6,10.0,6.0,0.6,10.0,20.0,0.6,33.333333,1.0,1.0,1.0
764153,Omanyte,26,680,97,73,181,160,101,68,0,Charizard,52,2389,399,369,337,468,376,440,0,Sunshine,-399,138,Rock,Water,6,Fire,Flying,0.0,-1.0,-1.0,1.0,1.0,1.0,-1.0,0.0,-1.0,1.0,2.0,0.0,0.0,0.0,-1.0,0.0,-1.0,0.0,-1.0,0.0,-1.0,-1.0,1.0,-1.0,-2.0,0.216617,2.038674,0.425532,4.633663,6.0,0.6,10.0,6.0,0.6,10.0,20.0,0.6,33.333333,1.0,1.0,1.0
764154,Omanyte,96,3417,551,356,898,782,494,336,0,Charizard,14,765,129,118,108,151,119,140,0,Unknown,551,138,Rock,Water,6,Fire,Flying,0.0,-1.0,-1.0,1.0,1.0,1.0,-1.0,0.0,-1.0,1.0,2.0,0.0,0.0,0.0,-1.0,0.0,-1.0,0.0,-1.0,0.0,-1.0,-2.0,0.0,1.0,-3.0,3.296296,0.131403,6.571429,0.305668,6.0,0.6,10.0,6.0,0.6,10.0,20.0,0.6,33.333333,1.0,1.0,1.0
764155,Omanyte,40,927,147,98,243,214,134,91,0,Charizard,84,4878,818,754,684,954,767,901,0,Rain,-818,138,Rock,Water,6,Fire,Flying,0.0,-1.0,-1.0,1.0,1.0,1.0,-1.0,0.0,-1.0,1.0,2.0,0.0,0.0,0.0,-1.0,0.0,-1.0,0.0,-1.0,0.0,1.0,-1.0,0.0,0.0,0.0,0.143275,3.102881,0.279009,7.119403,6.0,0.6,10.0,6.0,0.6,10.0,20.0,0.6,33.333333,1.0,1.0,1.0


In [136]:
print(battle_results_with_weakness_4.isnull().sum().sum())

0


In [137]:
print(len(battle_results))
print(len(preprocessed_battle_results))
print(len(all_pokemons))
print('With Types: ', len(battle_results_with_types))
print(len(battle_results_AD))
print(len(battle_results_with_weakness))
print(len(battle_results_with_weakness_4))

#Super cool!

2719163
2719163
151
With Types:  2719163
2719163
2719163
2719163


In [141]:
battle_results_with_weakness_4.loc[battle_results_with_weakness_4['Name_1']=='Mew']

#Legendary seems to have no effect on Damage calculation (observed from Internet)

,Name_1,Level_1,Price_1,HP_1,Attack_1,Defense_1,Sp_Atk_1,Sp_Def_1,Speed_1,Legendary_1,Name_2,Level_2,Price_2,HP_2,Attack_2,Defense_2,Sp_Atk_2,Sp_Def_2,Speed_2,Legendary_2,WeatherAndTime,BattleResult,ID_1,Type_1_1,Type_2_1,ID_2,Type_1_2,Type_2_2,Type_1_1_a,Type_1_1_b,Type_1_1_c,Type_1_1_d,Type_1_1_e,Type_1_2_a,Type_1_2_b,Type_1_2_c,Type_1_2_d,Type_1_2_e,Type_2_1_a,Type_2_1_b,Type_2_1_c,Type_2_1_d,Type_2_1_e,Type_2_2_a,Type_2_2_b,Type_2_2_c,Type_2_2_d,Type_2_2_e,WeatherAndTime_a,WeatherAndTime_b,WeatherAndTime_c,WeatherAndTime_d,WeatherAndTime_e,A1/D2,A2/D1,Sp_A1/Sp_D2,Sp_A2/Sp_D1,value_1_1ab,value_1_1ba,Eff_11,value_1_2ab,value_2_1ba,Eff_12,value_2_1ab,value_1_2ba,Eff_21,value_2_2ab,value_2_2ba,Eff_22
146665,Mew,29,1120,205,183,183,183,183,183,0,Pinsir,38,1339,214,332,259,140,179,215,0,Sunshine,-214,151,Psychic,0,127,Bug,0,-1.0,-2.0,-2.0,0.0,0.0,0.0,0.0,-1.0,0.0,0.0,0.0,0.0,0.0,0.0,-1.0,0.0,0.0,0.0,0.0,-1.0,-1.0,-1.0,1.0,-1.0,-2.0,0.706564,1.814208,1.022346,0.765027,1.0,6.0,0.166667,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0
146666,Mew,49,1779,329,290,290,290,290,290,0,Pinsir,84,4322,722,1063,833,444,573,687,0,Rain,-722,151,Psychic,0,127,Bug,0,-1.0,-2.0,-2.0,0.0,0.0,0.0,0.0,-1.0,0.0,0.0,0.0,0.0,0.0,0.0,-1.0,0.0,0.0,0.0,0.0,-1.0,1.0,-1.0,0.0,0.0,0.0,0.348139,3.665517,0.506108,1.531034,1.0,6.0,0.166667,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0
146667,Mew,34,1297,237,212,212,212,212,212,0,Pinsir,83,4175,697,1027,805,429,553,664,0,Unknown,-697,151,Psychic,0,127,Bug,0,-1.0,-2.0,-2.0,0.0,0.0,0.0,0.0,-1.0,0.0,0.0,0.0,0.0,0.0,0.0,-1.0,0.0,0.0,0.0,0.0,-1.0,-1.0,-2.0,0.0,1.0,-3.0,0.263354,4.844340,0.383363,2.023585,1.0,6.0,0.166667,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0
146668,Mew,83,4253,798,691,691,691,691,691,0,Pinsir,92,5151,861,1267,993,529,683,818,0,Unknown,-861,151,Psychic,0,127,Bug,0,-1.0,-2.0,-2.0,0.0,0.0,0.0,0.0,-1.0,0.0,0.0,0.0,0.0,0.0,0.0,-1.0,0.0,0.0,0.0,0.0,-1.0,-1.0,-2.0,0.0,1.0,-3.0,0.695871,1.833575,1.011713,0.765557,1.0,6.0,0.166667,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0
146669,Mew,18,850,155,139,139,139,139,139,0,Pinsir,83,4175,697,1027,805,429,553,664,0,Unknown,-697,151,Psychic,0,127,Bug,0,-1.0,-2.0,-2.0,0.0,0.0,0.0,0.0,-1.0,0.0,0.0,0.0,0.0,0.0,0.0,-1.0,0.0,0.0,0.0,0.0,-1.0,-1.0,-2.0,0.0,1.0,-3.0,0.172671,7.388489,0.251356,3.086331,1.0,6.0,0.166667,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2598961,Mew,91,5490,1030,892,892,892,892,892,0,Slowpoke,33,725,217,148,148,91,91,30,0,Unknown,1030,151,Psychic,0,79,Water,Psychic,-1.0,-2.0,-2.0,0.0,0.0,2.0,0.0,0.0,0.0,-1.0,0.0,0.0,0.0,0.0,-1.0,-1.0,-2.0,-2.0,0.0,0.0,-1.0,-2.0,0.0,1.0,-3.0,6.027027,0.165919,9.802198,0.102018,1.0,1.0,1.000000,0.6,0.6,1.0,1.0,1.0,1.0,1.0,1.0,1.0
2598962,Mew,80,3859,724,627,627,627,627,627,0,Slowpoke,39,859,257,176,176,107,107,36,0,Rain,724,151,Psychic,0,79,Water,Psychic,-1.0,-2.0,-2.0,0.0,0.0,2.0,0.0,0.0,0.0,-1.0,0.0,0.0,0.0,0.0,-1.0,-1.0,-2.0,-2.0,0.0,0.0,1.0,-1.0,0.0,0.0,0.0,3.562500,0.280702,5.859813,0.170654,1.0,1.0,1.000000,0.6,0.6,1.0,1.0,1.0,1.0,1.0,1.0,1.0
2598963,Mew,61,2376,441,387,387,387,387,387,0,Slowpoke,1,315,90,65,65,40,40,15,0,Sunshine,441,151,Psychic,0,79,Water,Psychic,-1.0,-2.0,-2.0,0.0,0.0,2.0,0.0,0.0,0.0,-1.0,0.0,0.0,0.0,0.0,-1.0,-1.0,-2.0,-2.0,0.0,0.0,-1.0,-1.0,1.0,-1.0,-2.0,5.953846,0.167959,9.675000,0.103359,1.0,1.0,1.000000,0.6,0.6,1.0,1.0,1.0,1.0,1.0,1.0,1.0
2598964,Mew,88,5071,951,824,824,824,824,824,0,Slowpoke,4,348,100,72,72,44,44,16,0,Sunshine,951,151,Psychic,0,79,Water,Psychic,-1.0,-2.0,-2.0,0.0,0.0,2.0,0.0,0.0,0.0,-1.0,0.0,0.0,0.0,0.0,-1.0,-1.0,-2.0,-2.0,0.0,0.0,-1.0,-1.0,1.0,-1.0,-2.0,11.444444,0.087379,18.727273,0.053398,1.0,1.0,1.000000,0.6,0.6,1.0,1.0,1.0,1.0,1.0,1.0,1.0


In [140]:
# preprocessed_battle_results.loc[preprocessed_battle_results['Name_1']=='Mew']

In [142]:
# battle_results_with_weakness_2.loc[battle_results_with_weakness_2['Type_2_1'] == '0']

In [260]:
print(weakness_long)
print(18**2)

       Type_1  Type_2  value
0      Normal  Normal    1.0
1        Fire  Normal    1.0
2       Water  Normal    1.0
3    Electric  Normal    1.0
4       Grass  Normal    1.0
..        ...     ...    ...
319     Ghost   Fairy    1.0
320    Dragon   Fairy    0.6
321      Dark   Fairy    0.6
322     Steel   Fairy    6.0
323     Fairy   Fairy    1.0

[324 rows x 3 columns]
324


In [143]:
def create_battle_results_with_weakness(
    battle_results: pd.DataFrame, weakness_pokemons: pd.DataFrame) -> pd.DataFrame:
    """Adds addtitional features for 'Effectiveness of Attacks' ('Eff_11', 'Eff_12', 'Eff_21', Eff_22'). 
       These are based on weakness ratio between the Pokemon's types. The numbers refer only to the types
       as the Pokemon can be omitted (ratio is 1/Eff) for the other => there is no new information.
       
       Notation: 11 => Type_1_1 : Type_1_2 (this means Type1 of Pokemon1 divided by Type1 of Pokemon2)
       
       
       When there is no Type 2 => Set Eff_12 (or Eff_21) to 1 as it should be neutral.
       
       Final Effectiveness is Eff = Eff_11 * Eff_12 * Eff_21 * Eff_22
       
       
        Args:
            battle_results: Preprocessed data for Battle Results
            weakness_pokemons: Preprocessed data for Weakness Pokemons.
        Returns:
            battle_results: Battle Results with Effectiveness

    """
    #Unpivot pokemon_weakness to Table with Type1, Type2 and value as columns
    weakness_long = weakness_pokemons.melt(id_vars="Types")
    weakness_long.columns = ["Type_P1", "Type_P2", "value"]

    #Eff_11
    battle_results_with_weakness = battle_results.merge(weakness_long, left_on=["Type_1_1","Type_1_2"], right_on=["Type_P1","Type_P2"])
    battle_results_with_weakness.rename(columns={'value': 'value_1_1ab'}, inplace=True)
    battle_results_with_weakness.drop(['Type_P1', 'Type_P2'], axis=1, inplace=True)

    battle_results_with_weakness = battle_results_with_weakness.merge(weakness_long, left_on=["Type_1_2","Type_1_1"], right_on=["Type_P1","Type_P2"])
    battle_results_with_weakness.rename(columns={'value': 'value_1_1ba'}, inplace=True)
    battle_results_with_weakness.drop(['Type_P1', 'Type_P2'], axis=1, inplace=True)

    battle_results_with_weakness = battle_results_with_weakness.fillna(1)

    battle_results_with_weakness['Eff_11'] = battle_results_with_weakness['value_1_1ab']/battle_results_with_weakness['value_1_1ba']

    
    #Eff_12
    battle_results_with_weakness_2 = battle_results_with_weakness.merge(weakness_long, left_on=["Type_1_1","Type_2_2"], right_on=["Type_P1","Type_P2"], how='left')
    battle_results_with_weakness_2.rename(columns={'value': 'value_1_2ab'}, inplace=True)
    battle_results_with_weakness_2.drop(['Type_P1', 'Type_P2'], axis=1, inplace=True)

    battle_results_with_weakness_2 = battle_results_with_weakness_2.merge(weakness_long, left_on=["Type_2_2","Type_1_1"], right_on=["Type_P1","Type_P2"], how='left')
    battle_results_with_weakness_2.rename(columns={'value': 'value_2_1ba'}, inplace=True)
    battle_results_with_weakness_2.drop(['Type_P1', 'Type_P2'], axis=1, inplace=True)

    battle_results_with_weakness_2 = battle_results_with_weakness_2.fillna(1)

    battle_results_with_weakness_2['Eff_12'] = battle_results_with_weakness_2['value_1_2ab']/battle_results_with_weakness_2['value_2_1ba']

    
    #Eff_21
    battle_results_with_weakness_3 = battle_results_with_weakness_2.merge(weakness_long, left_on=["Type_2_1","Type_1_2"], right_on=["Type_P1","Type_P2"], how='left')
    battle_results_with_weakness_3.rename(columns={'value': 'value_2_1ab'}, inplace=True)
    battle_results_with_weakness_3.drop(['Type_P1', 'Type_P2'], axis=1, inplace=True)

    battle_results_with_weakness_3 = battle_results_with_weakness_3.merge(weakness_long, left_on=["Type_1_2","Type_2_1"], right_on=["Type_P1","Type_P2"], how='left')
    battle_results_with_weakness_3.rename(columns={'value': 'value_1_2ba'}, inplace=True)
    battle_results_with_weakness_3.drop(['Type_P1', 'Type_P2'], axis=1, inplace=True)

    battle_results_with_weakness_3 = battle_results_with_weakness_3.fillna(1)

    battle_results_with_weakness_3['Eff_21'] = battle_results_with_weakness_3['value_2_1ab']/battle_results_with_weakness_3['value_1_2ba']

    
    #Eff_22
    battle_results_with_weakness_4 = battle_results_with_weakness_3.merge(weakness_long, left_on=["Type_2_1","Type_2_2"], right_on=["Type_P1","Type_P2"], how='left')
    battle_results_with_weakness_4.rename(columns={'value': 'value_2_2ab'}, inplace=True)
    battle_results_with_weakness_4.drop(['Type_P1', 'Type_P2'], axis=1, inplace=True)

    battle_results_with_weakness_4 = battle_results_with_weakness_4.merge(weakness_long, left_on=["Type_2_2","Type_2_1"], right_on=["Type_P1","Type_P2"], how='left')
    battle_results_with_weakness_4.rename(columns={'value': 'value_2_2ba'}, inplace=True)
    battle_results_with_weakness_4.drop(['Type_P1', 'Type_P2'], axis=1, inplace=True)

    battle_results_with_weakness_4 = battle_results_with_weakness_4.fillna(1)

    battle_results_with_weakness_4['Eff_22'] = battle_results_with_weakness_4['value_2_2ab']/battle_results_with_weakness_4['value_2_2ba']
    
    
    #Final Effectiveness 'Eff'
    
    battle_results_with_weakness_4['Eff'] = battle_results_with_weakness_4['Eff_11'] * battle_results_with_weakness_4['Eff_12'] * battle_results_with_weakness_4['Eff_21'] * battle_results_with_weakness_4['Eff_22']
    
    
    return battle_results_with_weakness_4 #master_table

In [144]:
battle_results_with_weakness = create_battle_results_with_weakness(battle_results_AD, weakness_pokemons)

In [145]:
battle_results_with_weakness.head()

,Name_1,Level_1,Price_1,HP_1,Attack_1,Defense_1,Sp_Atk_1,Sp_Def_1,Speed_1,Legendary_1,Name_2,Level_2,Price_2,HP_2,Attack_2,Defense_2,Sp_Atk_2,Sp_Def_2,Speed_2,Legendary_2,WeatherAndTime,BattleResult,ID_1,Type_1_1,Type_2_1,ID_2,Type_1_2,Type_2_2,Type_1_1_a,Type_1_1_b,Type_1_1_c,Type_1_1_d,Type_1_1_e,Type_1_2_a,Type_1_2_b,Type_1_2_c,Type_1_2_d,Type_1_2_e,Type_2_1_a,Type_2_1_b,Type_2_1_c,Type_2_1_d,Type_2_1_e,Type_2_2_a,Type_2_2_b,Type_2_2_c,Type_2_2_d,Type_2_2_e,WeatherAndTime_a,WeatherAndTime_b,WeatherAndTime_c,WeatherAndTime_d,WeatherAndTime_e,A1/D2,A2/D1,Sp_A1/Sp_D2,Sp_A2/Sp_D1,value_1_1ab,value_1_1ba,Eff_11,value_1_2ab,value_2_1ba,Eff_12,value_2_1ab,value_1_2ba,Eff_21,value_2_2ab,value_2_2ba,Eff_22,Eff
0,Metapod,30,441,150,36,103,47,47,58,0,Pinsir,47,1589,262,393,306,164,211,253,0,Windy,-262,11,Bug,0,127,Bug,0,0.0,0.0,-1.0,0.0,0.0,0.0,0.0,-1.0,0.0,0.0,0.0,0.0,0.0,0.0,-1.0,0.0,0.0,0.0,0.0,-1.0,0.0,-1.0,0.0,0.0,0.0,0.117647,3.815534,0.222749,3.489362,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0
1,Metapod,15,297,92,26,73,33,33,40,0,Pinsir,89,4913,821,1208,947,504,652,781,0,Windy,-821,11,Bug,0,127,Bug,0,0.0,0.0,-1.0,0.0,0.0,0.0,0.0,-1.0,0.0,0.0,0.0,0.0,0.0,0.0,-1.0,0.0,0.0,0.0,0.0,-1.0,0.0,-1.0,0.0,0.0,0.0,0.027455,16.547945,0.050613,15.272727,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0
2,Metapod,97,2007,693,161,463,218,218,254,0,Pinsir,67,2477,414,610,477,254,328,394,0,Night,-132,11,Bug,0,127,Bug,0,0.0,0.0,-1.0,0.0,0.0,0.0,0.0,-1.0,0.0,0.0,0.0,0.0,0.0,0.0,-1.0,0.0,0.0,0.0,0.0,-1.0,-1.0,-1.0,-2.0,0.0,-1.0,0.337526,1.317495,0.664634,1.165138,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0
3,Metapod,62,882,303,70,205,96,96,112,0,Pinsir,37,1288,206,319,249,135,172,207,0,Sunshine,-110,11,Bug,0,127,Bug,0,0.0,0.0,-1.0,0.0,0.0,0.0,0.0,-1.0,0.0,0.0,0.0,0.0,0.0,0.0,-1.0,0.0,0.0,0.0,0.0,-1.0,-1.0,-1.0,1.0,-1.0,-2.0,0.281124,1.556098,0.558140,1.406250,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0
4,Metapod,85,1548,535,124,357,168,168,196,0,Pinsir,84,4322,722,1063,833,444,573,687,0,Unknown,-664,11,Bug,0,127,Bug,0,0.0,0.0,-1.0,0.0,0.0,0.0,0.0,-1.0,0.0,0.0,0.0,0.0,0.0,0.0,-1.0,0.0,0.0,0.0,0.0,-1.0,-1.0,-2.0,0.0,1.0,-3.0,0.148860,2.977591,0.293194,2.642857,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0


In [146]:
battle_results_with_weakness.loc[(battle_results_with_weakness['Type_2_1'] == 'Water') & (battle_results_with_weakness['Type_1_2'] == 'Fire')]

,Name_1,Level_1,Price_1,HP_1,Attack_1,Defense_1,Sp_Atk_1,Sp_Def_1,Speed_1,Legendary_1,Name_2,Level_2,Price_2,HP_2,Attack_2,Defense_2,Sp_Atk_2,Sp_Def_2,Speed_2,Legendary_2,WeatherAndTime,BattleResult,ID_1,Type_1_1,Type_2_1,ID_2,Type_1_2,Type_2_2,Type_1_1_a,Type_1_1_b,Type_1_1_c,Type_1_1_d,Type_1_1_e,Type_1_2_a,Type_1_2_b,Type_1_2_c,Type_1_2_d,Type_1_2_e,Type_2_1_a,Type_2_1_b,Type_2_1_c,Type_2_1_d,Type_2_1_e,Type_2_2_a,Type_2_2_b,Type_2_2_c,Type_2_2_d,Type_2_2_e,WeatherAndTime_a,WeatherAndTime_b,WeatherAndTime_c,WeatherAndTime_d,WeatherAndTime_e,A1/D2,A2/D1,Sp_A1/Sp_D2,Sp_A2/Sp_D1,value_1_1ab,value_1_1ba,Eff_11,value_1_2ab,value_2_1ba,Eff_12,value_2_1ab,value_1_2ba,Eff_21,value_2_2ab,value_2_2ba,Eff_22,Eff
752219,Omastar,70,2817,461,336,694,632,395,299,0,Rapidash,11,636,100,124,86,98,98,130,0,Night,461,139,Rock,Water,78,Fire,0,0.0,-1.0,-1.0,1.0,1.0,1.0,-1.0,0.0,-1.0,1.0,2.0,0.0,0.0,0.0,-1.0,0.0,0.0,0.0,0.0,-1.0,-1.0,-1.0,-2.0,0.0,-1.0,3.906977,0.178674,6.448980,0.248101,6.0,0.6,10.0,1.0,1.0,1.0,20.0,0.6,33.333333,1.0,1.0,1.0,333.333333
752220,Omastar,14,681,106,82,169,154,95,75,0,Rapidash,96,4700,858,878,611,710,710,933,0,Windy,-858,139,Rock,Water,78,Fire,0,0.0,-1.0,-1.0,1.0,1.0,1.0,-1.0,0.0,-1.0,1.0,2.0,0.0,0.0,0.0,-1.0,0.0,0.0,0.0,0.0,-1.0,0.0,-1.0,0.0,0.0,0.0,0.134206,5.195266,0.216901,7.473684,6.0,0.6,10.0,1.0,1.0,1.0,20.0,0.6,33.333333,1.0,1.0,1.0,333.333333
752221,Omastar,57,2154,346,259,532,484,303,230,0,Rapidash,93,4276,780,798,557,646,646,849,0,Windy,84,139,Rock,Water,78,Fire,0,0.0,-1.0,-1.0,1.0,1.0,1.0,-1.0,0.0,-1.0,1.0,2.0,0.0,0.0,0.0,-1.0,0.0,0.0,0.0,0.0,-1.0,0.0,-1.0,0.0,0.0,0.0,0.464991,1.500000,0.749226,2.132013,6.0,0.6,10.0,1.0,1.0,1.0,20.0,0.6,33.333333,1.0,1.0,1.0,333.333333
752222,Omastar,66,2501,410,298,616,561,350,266,0,Rapidash,39,1188,210,225,156,180,180,237,0,Windy,410,139,Rock,Water,78,Fire,0,0.0,-1.0,-1.0,1.0,1.0,1.0,-1.0,0.0,-1.0,1.0,2.0,0.0,0.0,0.0,-1.0,0.0,0.0,0.0,0.0,-1.0,0.0,-1.0,0.0,0.0,0.0,1.910256,0.365260,3.116667,0.514286,6.0,0.6,10.0,1.0,1.0,1.0,20.0,0.6,33.333333,1.0,1.0,1.0,333.333333
752223,Omastar,42,1476,236,178,364,332,209,157,0,Rapidash,57,1720,313,322,224,260,260,341,0,Rain,184,139,Rock,Water,78,Fire,0,0.0,-1.0,-1.0,1.0,1.0,1.0,-1.0,0.0,-1.0,1.0,2.0,0.0,0.0,0.0,-1.0,0.0,0.0,0.0,0.0,-1.0,1.0,-1.0,0.0,0.0,0.0,0.794643,0.884615,1.276923,1.244019,6.0,0.6,10.0,1.0,1.0,1.0,20.0,0.6,33.333333,1.0,1.0,1.0,333.333333
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
764152,Omanyte,13,485,64,53,131,117,72,48,0,Charizard,68,3392,569,524,476,663,533,627,0,Windy,-569,138,Rock,Water,6,Fire,Flying,0.0,-1.0,-1.0,1.0,1.0,1.0,-1.0,0.0,-1.0,1.0,2.0,0.0,0.0,0.0,-1.0,0.0,-1.0,0.0,-1.0,0.0,0.0,-1.0,0.0,0.0,0.0,0.111345,4.000000,0.219512,9.208333,6.0,0.6,10.0,6.0,0.6,10.0,20.0,0.6,33.333333,1.0,1.0,1.0,3333.333333
764153,Omanyte,26,680,97,73,181,160,101,68,0,Charizard,52,2389,399,369,337,468,376,440,0,Sunshine,-399,138,Rock,Water,6,Fire,Flying,0.0,-1.0,-1.0,1.0,1.0,1.0,-1.0,0.0,-1.0,1.0,2.0,0.0,0.0,0.0,-1.0,0.0,-1.0,0.0,-1.0,0.0,-1.0,-1.0,1.0,-1.0,-2.0,0.216617,2.038674,0.425532,4.633663,6.0,0.6,10.0,6.0,0.6,10.0,20.0,0.6,33.333333,1.0,1.0,1.0,3333.333333
764154,Omanyte,96,3417,551,356,898,782,494,336,0,Charizard,14,765,129,118,108,151,119,140,0,Unknown,551,138,Rock,Water,6,Fire,Flying,0.0,-1.0,-1.0,1.0,1.0,1.0,-1.0,0.0,-1.0,1.0,2.0,0.0,0.0,0.0,-1.0,0.0,-1.0,0.0,-1.0,0.0,-1.0,-2.0,0.0,1.0,-3.0,3.296296,0.131403,6.571429,0.305668,6.0,0.6,10.0,6.0,0.6,10.0,20.0,0.6,33.333333,1.0,1.0,1.0,3333.333333
764155,Omanyte,40,927,147,98,243,214,134,91,0,Charizard,84,4878,818,754,684,954,767,901,0,Rain,-818,138,Rock,Water,6,Fire,Flying,0.0,-1.0,-1.0,1.0,1.0,1.0,-1.0,0.0,-1.0,1.0,2.0,0.0,0.0,0.0,-1.0,0.0,-1.0,0.0,-1.0,0.0,1.0,-1.0,0.0,0.0,0.0,0.143275,3.102881,0.279009,7.119403,6.0,0.6,10.0,

In [147]:
#Test lengths
print(len(battle_results))
print(len(preprocessed_battle_results))
print(len(all_pokemons))
print('With Types: ', len(battle_results_with_types))
print(len(battle_results_AD))
print(len(battle_results_with_weakness))
print(len(battle_results_with_weakness_4))

2719163
2719163
151
With Types:  2719163
2719163
2719163
2719163


In [ ]:
#Garbage

#     weakness_long = weakness_pokemons.melt(id_vars="Types")
#     weakness_long.columns = ["Type_1", "Type_2", "value"]
    
#     battle_results_AD.merge(weakness_long, left_on=["Type_1_1","Type_1_2"], right_on=["Type_1","Type_2"])
#     battle_results['Eff1_1'] = 
    
#     battle_results.drop(['Type_1', 'Type_2'] axis=1, inplace=True)
#     battle_results['Eff']
    
#     battle_results.rename(columns={'value': 'Type_1_1')
                                   
# submissions = catalog.load("all_pokemons")

